## **Importación de librerías**

In [12]:
# Tratamiento de datos.
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Visualizaciones.
import matplotlib.pyplot as plt
import seaborn as sns

## **Carga de datos `dataset_estudiantes.csv`**

Se importa el fichero `dataset_estudiantes` en un DataFrame utilizando `read_csv`.

In [13]:
df = pd.read_csv("../data/1.raw/dataset_estudiantes.csv")

## **Imputación de valores nulos**

#### **Variables categóricas**

In [14]:
col_hora = 'horario_estudio_preferido'
probs_hora = df[col_hora].value_counts(normalize=True)
fill_hora = df.index[df[col_hora].isna()] 
df.loc[fill_hora, col_hora] = np.random.choice(probs_hora.index, size=len(fill_hora), p=probs_hora.values)
print(f"El número de valores nulos en {col_hora} ahora es: {df[col_hora].isnull().sum()}","\n")
print(df[col_hora].value_counts(normalize=True))

El número de valores nulos en horario_estudio_preferido ahora es: 0 

horario_estudio_preferido
Tarde     0.386
Noche     0.377
Mañana    0.237
Name: proportion, dtype: float64


Se selecciona la columna `horario_estudio_preferido` y se calcula la proporción de cada categoría a partir de los valores observados. Dado que las categorías Noche (0,386) y Tarde (0,371) presentan frecuencias muy similares, se opta por un método de imputación que respete dicha distribución. En este contexto, imputar mediante la moda podría incrementar artificialmente la categoría más frecuente y modificar la composición del conjunto de datos, introduciendo un sesgo en análisis posteriores.

A continuación, se localizan las filas donde dicha variable presenta valores nulos y se imputan sustituyéndolos por categorías elegidas de forma aleatoria mediante **np.random.choice**, utilizando como probabilidades las proporciones observadas en el conjunto de datos, de modo que la imputación mantiene aproximadamente la distribución original, evitando introducir sesgos hacia una categoría concreta.

Por último, se verifica el resultado mostrando el número de valores nulos restantes y la distribución final de la variable, para comprobar que la imputación se ha realizado correctamente y que mantiene la distribución inicial.

In [15]:
col_estilo = 'estilo_aprendizaje'
probs_estilo = df[col_estilo].value_counts(normalize=True)
fill_estilo = df.index[df[col_estilo].isna()] 
df.loc[fill_estilo, col_estilo] = np.random.choice(probs_estilo.index, size=len(fill_estilo), p=probs_estilo.values)
print(f"El número de valores nulos en {col_hora} ahora es: {df[col_hora].isnull().sum()}","\n")
print(df[col_estilo].value_counts(normalize=True))

El número de valores nulos en horario_estudio_preferido ahora es: 0 

estilo_aprendizaje
Visual               0.385
Auditivo             0.266
Kinestésico          0.187
Lectura/Escritura    0.162
Name: proportion, dtype: float64


En este caso se aplica el mismo procedimiento de imputación en la variable `estilo_aprendizaje`, rellenando los valores nulos mediante un muestreo aleatorio basado en las proporciones observadas de sus categorías, con el objetivo de conservar su distribución original y evitar sesgos en el conjunto de datos.

In [16]:
num_cols = df.select_dtypes(include='number').columns
X = df[num_cols]
imputer = IterativeImputer(max_iter=10,random_state=42,estimator=None,tol=1e-3)
X_imputed = imputer.fit_transform(X)

df[num_cols] = pd.DataFrame(X_imputed, columns=num_cols, index=df.index)
df['horas_sueno'] = df['horas_sueno'].clip(4, 10)

print(f"El número de valores nulos en horas_sueno ahora es:", df['horas_sueno'].isna().sum())

El número de valores nulos en horas_sueno ahora es: 0


Se aplica una imputación mediante **IterativeImputer** para completar los valores faltantes de `horas_sueno`, ya que se trata de una variable numérica continua con alta variabilidad y un gran número de valores únicos, por lo que imputar con una medida fija como media o mediana podría simplificar en exceso la información. 

Este método permite estimar los valores ausentes utilizando la relación de `horas_sueno` con el resto de variables numéricas del conjunto de datos, primero se seleccionan las columnas numéricas relevantes, después se configura el imputador y se ajusta el modelo con **fit_transform** para generar las imputaciones de forma iterativa, refinando las estimaciones en cada ciclo. 

Finalmente, se sustituyen los valores imputados en el DataFrame y se verifica el resultado comprobando que el número de valores nulos en la variable se ha reducido a cero y que la variable mantiene una distribución coherente.